In [21]:
import numpy as np
from numba import njit, prange
import time
from scipy.stats import linregress
import cupy as cp

# NUMBA

In [22]:
TRANSFORMATIONS = np.array([
    [[0, -1], [1, 0]],     # 90° Rotation
    [[-1, 0], [0, -1]],    # 180° Rotation
    [[0, 1], [-1, 0]],     # 270° Rotation
    [[1, 0], [0, -1]],     # Reflection x-axis
    [[-1, 0], [0, 1]],     # Reflection y-axis
    [[0, 1], [1, 0]],      # Reflection x=y
    [[0, -1], [-1, 0]],    # Reflection x=-y
], dtype=np.float64)

@njit
def initialize_saws(L):
    saws = np.zeros((4, L+1, 2), dtype=np.float64)
    for i in range(L+1):
        saws[0, i] = [0, i]     # Up
        saws[1, i] = [0, -i]    # Down
        saws[2, i] = [i, 0]     # Right
        saws[3, i] = [-i, 0]    # Left
    return saws

@njit
def apply_transformation(walk, pivot_index, transformation_matrix):
    new_walk = walk.copy()
    pivot_point = walk[pivot_index]
    for i in range(pivot_index + 1, len(walk)):
        rel = walk[i] - pivot_point
        new_walk[i] = (transformation_matrix @ rel) + pivot_point
    return new_walk

@njit
def check_self_avoidance(walk):
    seen = set()
    for i in range(walk.shape[0]):
        key = (int(walk[i, 0]), int(walk[i, 1]))
        if key in seen:
            return False
        seen.add(key)
    return True

@njit(parallel=True)
def monte_carlo_parallel(num_samples, L):
    saws = initialize_saws(L)
    samples = np.zeros((num_samples, L+1, 2), dtype=np.float64)
    # pivots = np.full(num_samples, -1, dtype=np.int32)  # default -1 = initial
    # transformations = np.full(num_samples, -1, dtype=np.int32)  # default -1 = initial

    per_chain = num_samples // 4

    for chain in prange(4):
        current_walk = saws[chain].copy()
        count = 0
        index_base = chain * per_chain

        samples[index_base + count] = current_walk
        # pivots[index_base + count] = -1
        # transformations[index_base + count] = -1
        count += 1

        while count < per_chain:
            pivot = np.random.randint(1, L)
            t_idx = np.random.randint(0, len(TRANSFORMATIONS))
            T = TRANSFORMATIONS[t_idx]
            transformed_walk = apply_transformation(current_walk, pivot, T)

            if check_self_avoidance(transformed_walk):
                current_walk = transformed_walk
                samples[index_base + count] = current_walk
                # pivots[index_base + count] = pivot
                # transformations[index_base + count] = t_idx
                count += 1

    return samples

In [23]:
@njit
def concatenate_saws(walk1, walk2):
    """Translate walk2 so that it starts where walk1 ends, then concatenate."""
    endpoint = walk1[-1]  # Get the last point of walk1
    translation_vector = endpoint - walk2[0]  # Calculate how much to shift walk2
    translated_walk2 = walk2 + translation_vector  # Apply the shift

    # Concatenate the translated second walk onto the first
    concatenated_walk = np.vstack((walk1, translated_walk2[1:]))

    return concatenated_walk

In [24]:
@njit(parallel = True)
def compute_B(samples):
    """Estimate B(L1, L2) by checking valid concatenations of independent SAWs."""
    valid = 0
    N = len(samples)
    total = N**2  # Include self-pairing cases

    for i in prange(N):
        for j in range(N):  # Every walk can pair with every other walk, including itself
            concatenated = concatenate_saws(samples[i], samples[j])
            if check_self_avoidance(concatenated):
                valid += 1

    return valid / total

In [25]:
@njit(parallel = True)
def recursive(initial_L, initial_c_L, iterations, N):
    L = initial_L
    cL = initial_c_L
    count = 1

    Ls = np.zeros(iterations+1, dtype = np.float64)
    cLs = np.zeros(iterations+1, dtype = np.float64)

    Ls[0] = L
    cLs[0] = cL
    
    while count <= iterations:
        samples = monte_carlo_parallel(N, L)
        proportion = compute_B(samples)
        cL = proportion * (cL ** 2)
        L = 2*L
        
        Ls[count] = L
        cLs[count] = cL
        count += 1
    return cLs, Ls

In [27]:
def estimate_mu(Ls, cLs):
    x = 1/Ls
    y = np.log(cLs) / Ls

    slope, intercept, r_val, p_val, std_err = linregress(x,y)

    mu = np.exp(intercept)
    return mu

In [26]:
@njit(parallel = True)
def large_recursive(initial_L, initial_c_L, iterations, N):
    L = initial_L
    log_cL = np.log(initial_c_L)
    count = 1

    Ls = np.zeros(iterations+1, dtype = np.float64)
    cLs = np.zeros(iterations+1, dtype = np.float64)

    Ls[0] = L
    cLs[0] = log_cL
    
    while count <= iterations:
        samples = monte_carlo_parallel(N, L)
        proportion = compute_B(samples)
        log_cL = np.log(proportion) +  (2 * log_cL)
        L = 2*L
        
        Ls[count] = L
        cLs[count] = log_cL
        count += 1
    return cLs, Ls

In [32]:
def ln_estimate_mu(Ls, log_cLs):
    x = 1/Ls
    y = log_cLs / Ls

    slope, intercept, r_val, p_val, std_err = linregress(x,y)

    mu = np.exp(intercept)
    return mu

In [16]:
cLs, Ls = recursive(20, 897697164, 7, 10**4)

In [17]:
cLs

array([8.97697164e+008, 3.02067541e+017, 2.81685551e+034, 1.97789968e+068,
       7.54999177e+135, 9.07107193e+270,             inf,             inf])

In [18]:
Ls

array([  20.,   40.,   80.,  160.,  320.,  640., 1280., 2560.])

In [22]:
cLs[:6], Ls[:6]

(array([8.97697164e+008, 3.02067541e+017, 2.81685551e+034, 1.97789968e+068,
        7.54999177e+135, 9.07107193e+270]),
 array([ 20.,  40.,  80., 160., 320., 640.]))

In [25]:
estimate_mu(Ls[:6], cLs[:6])

np.float64(2.651804267998532)

In [28]:
log_cLs, Ls = large_recursive(20, 897697164, 7, 10**4)

In [29]:
# 2044 seconds ^

In [30]:
log_cLs

array([  20.61534334,   40.23167072,   79.28838392,  157.13976894,
        312.62346889,  623.29698943, 1244.34364318, 2486.47603756])

In [33]:
ln_estimate_mu(Ls, log_cLs)

np.float64(2.646086666480796)

# CuPy

In [34]:
import cupy as cp

TRANSFORMATIONS = cp.array([
    [[0, -1], [1, 0]],     # 90° Rotation
    [[-1, 0], [0, -1]],    # 180° Rotation
    [[0, 1], [-1, 0]],     # 270° Rotation
    [[1, 0], [0, -1]],     # Reflection x-axis
    [[-1, 0], [0, 1]],     # Reflection y-axis
    [[0, 1], [1, 0]],      # Reflection x=y
    [[0, -1], [-1, 0]],    # Reflection x=-y
], dtype=cp.float64)

def monte_carlo_parallel_gpu(num_samples, L):
    """Generate SAW samples using CuPy on GPU."""
    saws = cp.zeros((4, L + 1, 2), dtype=cp.float64)

    # Initialize base walks for 4 directions
    for i in range(L + 1):
        saws[0, i] = [0, i]    # Up
        saws[1, i] = [0, -i]   # Down
        saws[2, i] = [i, 0]    # Right
        saws[3, i] = [-i, 0]   # Left

    samples = cp.zeros((num_samples, L + 1, 2), dtype=cp.float64)
    per_chain = num_samples // 4

    for chain in range(4):
        current_walk = cp.array(saws[chain])
        count = 0
        index_base = chain * per_chain

        while count < per_chain:
            pivot = cp.random.randint(1, L)
            t_idx = cp.random.randint(0, len(TRANSFORMATIONS))
            T = TRANSFORMATIONS[t_idx]
            transformed_walk = apply_transformation_gpu(current_walk, pivot, T)

            if check_self_avoidance_gpu(transformed_walk):
                current_walk = transformed_walk
                samples[index_base + count] = current_walk
                count += 1

    return samples


def compute_B_gpu(samples):
    """Estimate B(L, L) using CuPy tensor operations."""
    N = samples.shape[0]
    valid_pairs = cp.zeros((N, N), dtype=cp.float64)

    for i in range(N):
        for j in range(N):
            concatenated = cp.vstack((samples[i], samples[j][1:]))
            if check_self_avoidance_gpu(concatenated):
                valid_pairs[i, j] = 1

    return cp.mean(valid_pairs).item()


def apply_transformation_gpu(walk, pivot_index, transformation_matrix):
    """Apply a transformation to a walk using CuPy tensors."""
    pivot_point = walk[pivot_index]
    rel_positions = walk[pivot_index + 1:] - pivot_point
    new_positions = cp.dot(transformation_matrix, rel_positions.T).T + pivot_point
    new_walk = walk.copy()
    new_walk[pivot_index + 1:] = new_positions
    return new_walk


def check_self_avoidance_gpu(walk):
    """Check if a walk avoids itself using CuPy."""
    seen = set()
    for point in walk:
        key = tuple(point.tolist())
        if key in seen:
            return False
        seen.add(key)
    return True


def large_recursive_gpu(initial_L, initial_c_L, iterations, N):
    """Fully GPU-accelerated recursive computation."""
    L = initial_L
    log_cL = cp.log(initial_c_L)
    Ls = cp.zeros(iterations + 1, dtype=cp.float64)
    log_cLs = cp.zeros(iterations + 1, dtype=cp.float64)

    Ls[0] = L
    log_cLs[0] = log_cL

    for count in range(1, iterations + 1):
        samples = monte_carlo_parallel_gpu(N, L)
        proportion = compute_B_gpu(samples)
        if proportion == 0:
            proportion = 1e-300  # Avoid log(0)
        log_cL = cp.log(proportion) + (2 * log_cL)
        L = 2 * L

        Ls[count] = L
        log_cLs[count] = log_cL

    return cp.asnumpy(log_cLs), cp.asnumpy(Ls)  # Convert to NumPy for plotting or further analysis

In [35]:
large_recursive_gpu(20, 897697164, 7, 10**4)

TypeError: Unsupported type <class 'list'>